In [6]:
import numpy as np
import math
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import truncnorm
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb
#
#
import nevergrad as ng
# import autograd.numpy as au
# from autograd import grad, jacobian
#
from concurrent import futures
import time
import os
#
from scipy.optimize import Bounds
from scipy.optimize import minimize
#
import ast

## Setup

In [56]:
np.random.RandomState(1234)
np.random.seed(42)
n_list = [10,15]
m_list = [5]
budget = {'default':20}#, (10,5):200, (15,5):150, (15,10):100}
maxiter = 350
K = 35
constraint_tolerance = .01
opt_list = [ng.optimizers.NGOpt, ng.optimizers.NGOptRW]#, ng.optimizers.NelderMead, ng.optimizers.Cobyla]
filename = "results_rw_n15_m10.json"

In [57]:
def rosenbrock(x, n):
    sum = 0.0
    for i in range(n-1):
        a = 100 * ((x[i+1] -x[i]**2)**2)
        b = (1 - x[i])**2
        sum += a+b
    return sum

def get_truncated_normal(mean=5, sd=1, low=0, upp=10, size=None):
    """normal distribution between 0 and 10"""
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs(size)

In [58]:
def get_random_constants(k:int, mean=5., sd=1., seed=42, low=0, upp=10):
    np.random.seed(seed)
    return get_truncated_normal(size=k, mean=mean, sd=sd, low=low, upp=upp)

def get_random_indices(n:int, seed:int=42):
    indices = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(indices)
    return indices

def fill_with_constants(x:np.array, constants:np.array, indices:np.array):
    #return np.append(x,constants)[indices]
    return np.take(np.append(x,constants), indices)

def fill_with_constants_tf(x, constants:np.array, indices:np.array):
    y = tf.concat([x,constants], axis=-1)
    return tf.gather(y, indices)

In [59]:
ann_models = {n: tf.keras.models.load_model(f"models/ANN_{n}") for n in n_list}
xgb_models = {n: xgb.Booster() for n in n_list}
for n in n_list:
    xgb_models[n].load_model(f"models/XGB_{n}.json")
#
with open('models/metadata.json', 'r') as file:
    metadata = json.load(file)
#
print("R^2 values:\n")
for n in metadata.keys():
    print(f"n={n}")
    print(f"ANN: {round(metadata[n]['ANN_R2'],4)}\nXGB: {round(metadata[n]['XGB_R2'],4)}")
    print("\n")
print(ann_models)
#print(xgb_models)

R^2 values:

n=10
ANN: -12.116
XGB: 0.9726


n=15
ANN: -18.0735
XGB: 0.9553


{10: <keras.engine.sequential.Sequential object at 0x7fb6d68ee730>, 15: <keras.engine.sequential.Sequential object at 0x7fb71403fcd0>}


## Models and objective functions

In [61]:
def relu(x):
    return np.maximum(0, x)


def get_functions(n, constants, indices):
    #
    weights = ann_models[n].get_weights()
    def f_ann(x):
        x1 = fill_with_constants(x, constants, indices)
        z1 = np.dot(x1, weights[0]) + weights[1]
        a1 = relu(z1)
        z2 = np.dot(a1, weights[2]) + weights[3]
        a2 = relu(z2)
        z3 = np.dot(a2, weights[4]) + weights[5]
        return z3
    #
    #grad_f_ann = grad(f_ann)
    #
    def f_ann_tf(x):
        x1 = fill_with_constants_tf(x, constants, indices)
        x2 = tf.convert_to_tensor([x1])
        x2 = tf.cast(x2,dtype=tf.float32)
        return ann_models[n](x2)[0]
    #
    def grad_f_ann_tf(x):
        with tf.GradientTape() as tape:
            tape.watch(x)
            t2 = f_ann_tf(x)
            y = tape.jacobian(t2, x)
            return y
        
    def hess_tf(x):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            t2 = f_ann_tf(x)
            y = tape.gradient(t2, x)
            h = tape.jacobian(y, x)
            return y, h
    #
    def f_xgb(x):
        x1 = fill_with_constants(x, constants, indices)
        y = xgb_models[n].predict(xgb.DMatrix([x1]))#x1.reshape(1,-1))
        return y.item()
    #
    def f_rosen(x):
        x1 = fill_with_constants(x, constants, indices)
        return rosenbrock(x1, n)
    #
    return f_rosen, f_ann_tf, grad_f_ann_tf, f_xgb, hess_tf
    #return f_rosen, f_ann, grad_f_ann, f_xgb

## Optimization methods

In [62]:
def optimize_ng(m, f, constraint_c, optclass, budget=1_000, x_start=None, constraint_tolerance=.1):
    instrum = ng.p.Instrumentation(
        ng.p.Array(shape=(m,)).set_bounds(lower=0.0, upper=10),
    )
    optimizer = optclass(parametrization=instrum, budget=budget, num_workers=1)
    optimizer.parametrization.register_cheap_constraint(lambda x:np.sum(x[0]) - constraint_c)
    optname = str(optclass).split(".")[-1].split("'")[0]
    filepath = f"optimization/{optname}_logs.txt"
    optimizer.register_callback("tell", ng.callbacks.ParametersLogger(filepath, append=False))
    #
    start = time.time()
    recommendation = optimizer.minimize(f, verbosity=0, batch_mode=False)
    #with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
    #    recommendation = optimizer.minimize(f, verbosity=0, executor=executor, batch_mode=False)
    end = time.time()
    elapsed_time = end - start
    #
    with open(filepath, 'r') as file:
        lines = file.readlines()
        log = [ast.literal_eval(line) for line in lines]
    #
    ## select argument with minimum function value satisfying the constraint
    x_list = [l['0'] for l in log]
    x_constrained_list = [x for x in x_list if np.abs((np.sum(x)-constraint_c))/constraint_c < constraint_tolerance]
    #x_constrained_list = x_constrained_list or x_list
    if not x_constrained_list:
        x_constrained_list = x_list
        print("Could not satisfy constraints.")
    loss_array = np.array([f(x) for x in x_constrained_list])
    min_index = loss_array.argmin()
    x_min = x_constrained_list[min_index]
    #
    #return np.asarray(recommendation.value)[0][0], elapsed_time
    return np.asarray(x_min), elapsed_time


def optimize_grad(m, f, grad_f, constraint_c, maxiter=maxiter, x_start=None):
    if x_start is None: x_start = np.ones(m)
    eq_cons = {'type': 'eq',
               'fun' : lambda x:np.sum(x)-constraint_c
               }
    bounds = Bounds([0.0]*m, [10.0]*m)
    start = time.time()
    result = minimize(fun=f, x0=x_start, jac=grad_f,constraints=[eq_cons], method='SLSQP',
    options={'ftol': 1e-12, 'disp': False, 'maxiter':maxiter}, bounds=bounds)
    end = time.time()
    #
    elapsed_time = end - start
    return result.x, elapsed_time

In [63]:
n=15
m=10
constants = get_random_constants(n-m, seed=seed)
indices = get_random_indices(n, seed=seed)

In [64]:
f_rosen, f_ann, grad_f_ann,f_xgb, hess_tf = get_functions(n, constants, indices)

In [65]:
hess_tf = tf.function(hess_tf)
hess_tf(tf.random.normal([10]))

(<tensorflow.python.framework.indexed_slices.IndexedSlices at 0x7fb6d6459970>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[-0.04190152, -0.04610078, -0.02778385, -0.02734154, -0.04667171,
         -0.0262934 , -0.02368741, -0.03795334, -0.02840973, -0.01509296],
        [-0.04610078, -0.09451991, -0.01778026, -0.03560251, -0.07787273,
         -0.04970959, -0.03674165, -0.04405955, -0.05158599, -0.01428949],
        [-0.02778385, -0.01778027, -0.0347028 , -0.01835832, -0.02983059,
         -0.01679232, -0.01171066, -0.01387203, -0.01398957, -0.01063736],
        [-0.02734154, -0.03560251, -0.01835832, -0.04595041, -0.03500346,
         -0.0279151 , -0.02116625, -0.03274539, -0.02161375, -0.00885629],
        [-0.0466717 , -0.07787275, -0.02983058, -0.03500344, -0.09491346,
         -0.04584476, -0.03788688, -0.05668491, -0.05215588, -0.0250662 ],
        [-0.02629339, -0.04970958, -0.01679232, -0.02791509, -0.04584478,
         -0.04117362, -0.03196042, -0.03522967, -0.

In [18]:
results = []
for n in n_list:
    m_list_current = [m for m in m_list if m < n]
    for m in m_list_current:
        for k in range(K):
            print(f"Optimizing for n={n} and m={m}, trial {k+1}/{K}")
            #
            seed = k + 1_000*n + 1_000_000*m
            results_current = {'n': n, 'm': m, 'k': k}
            constants = get_random_constants(n-m, seed=seed)
            indices = get_random_indices(n, seed=seed)
            constraint_c = 5*m
            current_budget = budget.get((n,m), budget['default'])
            #
            f_rosen, f_ann, grad_f_ann,f_xgb = get_functions(n, constants, indices)
            f_ann, grad_f_ann, hess_tf = tf.function(f_ann), tf.function(grad_f_ann)
            #
            x_min = {}
            for optclass in opt_list:
                optname = str(optclass).split(".")[-1].split("'")[0]
                results_current[optname] = {}
                x_min, elapsed_time = optimize_ng(m, f_xgb, constraint_c, optclass=optclass, budget=current_budget, constraint_tolerance=constraint_tolerance, x_start=None)
                y_min = f_rosen(x_min)
                results_current[optname]['x_min'] = x_min.tolist()
                results_current[optname]['time'] = elapsed_time
                results_current[optname]['loss'] = y_min
                results_current[optname]['budget'] = current_budget
            #
            x_min_ann, elapsed_time_ann = optimize_grad(m, f_ann, grad_f_ann, constraint_c)
            #
            y_min_ann = f_rosen(x_min_ann)
            #
            results_current['ann'] = {'loss': y_min_ann, 'time': elapsed_time_ann, 'x_min': x_min_ann.tolist()}
            results.append(results_current)
            #
            with open(os.path.join('optimization', filename), 'w') as file:
                json.dump(results, file)

Optimizing for n=15 and m=10, trial 1/35


/home/cican/miniconda3/lib/python3.9/site-packages/nevergrad/parametrization/core.py:332: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/lib/python3.9/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


Could not satisfy constraints.


/home/cican/miniconda3/lib/python3.9/site-packages/nevergrad/parametrization/core.py:332: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/lib/python3.9/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


TypeError: _minimize_trustregion_constr() got an unexpected keyword argument 'ftol'

## Data validation

In [ ]:
#print([r['NelderMead']['loss'] for r in results if r['n']==15 and r['m']==10])
#print([r['ann']['loss'] for r in results if r['n']==15 and r['m']==10])
#
#print([np.sum(r['NelderMead']['x_min']) for r in results if r['n']==15 and r['m']==10])
#print([np.sum(r['ann']['x_min']) for r in results if r['n']==15 and r['m']==10])
#
print([r['NelderMead']['loss'] for r in results if r['n']==10 and r['m']==5])
print([r['ann']['loss'] for r in results if r['n']==10 and r['m']==5])
#
print([np.sum(r['NelderMead']['x_min']) for r in results if r['n']==10 and r['m']==5])
print([np.sum(r['ann']['x_min']) for r in results if r['n']==10 and r['m']==5])

In [ ]:
k=K-1
n=15
m=10
seed = k + 1_000*n + 1_000_000*m
results_current = {'n': n, 'm': m, 'k': k}
constants = get_random_constants(n-m, seed=seed)
indices = get_random_indices(n, seed=seed)
constraint_c = 5*m
#
f_rosen, f_ann, grad_f_ann, f_xgb = get_functions(n, constants, indices)
f_ann, grad_f_ann = tf.function(f_ann), tf.function(grad_f_ann)
x_min_ann, elapsed_time_ann = optimize_grad(m, f_ann, grad_f_ann, constraint_c, maxiter=1e7)
#
y_min_ann = f_rosen(x_min_ann)
results_current['ann'] = {'loss': y_min_ann, 'time': elapsed_time_ann, 'x_min': x_min_ann.tolist()}
print(results_current)
print(results_current['ann']['loss'])
np.random.seed(16807)
print([np.sum(r['NelderMead']['x_min']) for r in results if r['m']==10 and r['n']==15])
print([r['NelderMead']['loss'] for r in results if r['m']==10 and r['n']==15])
print(f_rosen(x_min_ann))#+0.01*np.random.uniform(size=m)))
print(np.sum(x_min_ann))

In [ ]:
with open('optimization/NelderMead_logs.txt', 'r') as file:
    lines = file.readlines()
    log = []
    for line in lines:
        try:
            log.append(ast.literal_eval(line))
        except ValueError:
            print("malformed string; skipping this line")
        except SyntaxError:
            print("looks like some encoding errors with this file...")
#
#print([np.sum(l['0']) for l in log])
constraint_c = 50.
constraint_tolerance = .1
#
x_list = [l['0'] for l in log]
x_constrained_list = [x for x in x_list if np.abs((np.sum(x)-constraint_c))/constraint_c < constraint_tolerance]
print([f_rosen(x) for x in x_constrained_list])
loss_array = np.array([f_xgb(x) for x in x_constrained_list])
min_index = loss_array.argmin()
x_min_temp = x_constrained_list[min_index]
print(np.sum(x_min_temp))
print(f_rosen(x_min_temp))
#print(min([f_xgb(l['0']) for l in log]))